In [1]:
import json
import os
import pandas as pd
source = "../DATA"
from collections import defaultdict
import nltk 
nltk.download()

In [2]:
lang_data = json.load(open(f"{source}/clean_data.json"))
all_artcile_data = lang_data

In [4]:
article = all_artcile_data["bf1bd2dfb4e08868d7e55d1b6fe553761b2299a3"]
print(article)

{'paperId': 'bf1bd2dfb4e08868d7e55d1b6fe553761b2299a3', 'title': 'analysis of nonlinear photoconductive (nlp) switching in bulk gaas', 'abstract': 'this paper presents a numerical analysis of nonlinear photoconductive (nlp) switching in bulk gallium arsenide. nlp switches are optically activated switches in which the number of electron-hole pairs initially created are multiplied through a nonlinear process. this process is a combination of double injection, trap filling, and avalanche. nlp switches differ from linear photoconductive (lp) switches in which one photon creates one electron-hole pair. the numerical method used to solve the governing nlp switch equations and preliminary results from the computer model are presented.', 'year': 1991, 'authors': [{'authorId': '31165417', 'name': 'M. Browder'}, {'authorId': '5617715', 'name': 'W. Nunnally'}], 'fieldsOfStudy': ['materials science'], 'citationCount': 6, 'referenceCount': 3, 'references': [{'paperId': '90f811e04ab1df1f635bb511b5c4

In [6]:
i = iter(range(1,len(all_artcile_data)+1));
j = iter(range(1,len(all_artcile_data)+1));
doc_to_id = defaultdict(lambda : next(i));
author_to_id = defaultdict(lambda :[]);
list(map(lambda x : doc_to_id[x],all_artcile_data));
def put_author(item):
    global author_to_id
    for author in item:
        name = author["name"].lower()
        id = author["authorId"]
        author_to_id[name] = id
        try:
            id = int(id)
        except:
            id = id
        author_to_id[id].append(name)
        author_to_id[id] = list(set(author_to_id[id]))

[put_author(all_artcile_data[item]["authors"]) for item in all_artcile_data]
assert max(doc_to_id.values()) == len(all_artcile_data) , "size does not match"
open("DATA/P3/doc_to_integerID.json","w").write(json.dumps(doc_to_id))
open("DATA/P3/author_to_id.json","w").write(json.dumps(author_to_id))

1543827

In [7]:
author_to_doc = defaultdict(lambda :[])
for item in all_artcile_data:
    ar = all_artcile_data[item]
    for author in ar["authors"]:
        author_to_doc[author["authorId"]].append(item)
open("DATA/P3/author_to_doc.json","w").write(json.dumps(author_to_doc))  
i = iter(range(1,len(author_to_id)+1))
authorid_to_num_id = defaultdict(lambda : next(i))
list(map(lambda x : authorid_to_num_id[x],[key for key in author_to_id if type(key) == int]))
open("DATA/P3/authorid_to_num_id.json","w").write(json.dumps(authorid_to_num_id))

504850

In [8]:

authorid_to_num_id = defaultdict(lambda : next(i))

In [10]:
import spacy
nlp = spacy.load("en_core_web_sm")
flatten = lambda x : [item for subitem in x for item in subitem]

In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")
i = 0
def nx():
    global i
    i += 1
    return i
j = 1
def nx2():
    global j
    j += 1
    return j
title_lemma = defaultdict(lambda : nx())
abstract_lemma = defaultdict(lambda : nx2())
for key in all_artcile_data:
    article = all_artcile_data[key]
    if type(article["title"]) == str:
        title = article["title"].strip().lower()
        for w in nlp(title):
            title_lemma[w.lemma_]
    if type(article["abstract"]) == str:
        abstract = article["abstract"].strip().lower()
        for w in nlp(abstract):
            abstract_lemma[w.lemma_]
open("DATA/P3/title_lemma.json","w").write(json.dumps(title_lemma))
open("DATA/P3/abstract_lemma.json","w").write(json.dumps(abstract_lemma))

735170

In [12]:
bool_dic_title = defaultdict(lambda : [])
import numpy as np
removed_key = []
for key in all_artcile_data:
    article = all_artcile_data[key]
    if type(article["title"]) == str:
        title = article["title"].strip().lower()
        bool_dic_title[key] = [title_lemma[w.lemma_] for w in nlp(title)]
open("DATA/P3/bool_dic_title.json","w").write(json.dumps(bool_dic_title))

1068925

In [13]:
title_data = json.load(open("DATA/P3/title_lemma.json","r"))
abs_data = json.load(open("DATA/P3/abstract_lemma.json","r"))
import numpy as np
freq_matrix_title = np.zeros((len(all_artcile_data)+2,len(title_data)+2))
freq_matrix_abstract = np.zeros((len(all_artcile_data)+2,len(abs_data)+2))
n = 0
for key in all_artcile_data:
    n += 1
    article = all_artcile_data[key]
    if type(article["title"]) == str:
        title = article["title"].strip().lower()
        for w in nlp(title):
            if w.lemma_ not in nlp.Defaults.stop_words:
                freq_matrix_title[n][title_data[w.lemma_]] += 1
    if type(article["abstract"]) == str:
        abstract = article["abstract"].strip().lower()
        for w in nlp(abstract):
            if w.lemma_ not in nlp.Defaults.stop_words:
                freq_matrix_abstract[n][abs_data[w.lemma_]] += 1

In [14]:
np.savez_compressed("DATA/P3/matrix.npz", freq_matrix_title=freq_matrix_title,freq_matrix_abstract=freq_matrix_abstract)
v=np.load("DATA/P3/matrix.npz")
freq_matrix_title = v["freq_matrix_title"]
freq_matrix_abstract = v["freq_matrix_abstract"]

In [15]:
print(freq_matrix_title.shape)
print(freq_matrix_abstract.shape)

(10750, 9472)
(10750, 39056)


In [16]:
M = len(all_artcile_data)
sum_matrix = (freq_matrix_title > 0).sum(axis = 0)
idf_title = {int(title_data[key]) : (np.log2(M/sum_matrix[title_data[key]]) if sum_matrix[title_data[key]] != 0 else 0) for key in title_data}
M = len(all_artcile_data)
sum_matrix = (freq_matrix_abstract > 0).sum(axis = 0)
idf_abstract = {int(abs_data[key]) : (np.log2(M/sum_matrix[abs_data[key]]) if sum_matrix[abs_data[key]] != 0 else 0) for key in abs_data}
open("DATA/P3/idf_title.json","w").write(json.dumps(idf_title))
open("DATA/P3/idf_abstract.json","w").write(json.dumps(idf_abstract))

1108837

In [17]:
article_ids = [key for key in all_artcile_data]
len(article_ids)

10748

In [18]:
tf_title = np.log2(1+freq_matrix_title/freq_matrix_title.max(axis=1).reshape(-1,1))
tf_abs = np.log2(1+freq_matrix_abstract/freq_matrix_abstract.max(axis=1).reshape(-1,1))

/tmp/ipykernel_110726/3948576010.py:1: RuntimeWarning: invalid value encountered in true_divide
  tf_title = np.log2(1+freq_matrix_title/freq_matrix_title.max(axis=1).reshape(-1,1))
/tmp/ipykernel_110726/3948576010.py:2: RuntimeWarning: invalid value encountered in true_divide
  tf_abs = np.log2(1+freq_matrix_abstract/freq_matrix_abstract.max(axis=1).reshape(-1,1))


In [19]:
n = 0
title_tf = {}
asb_tf = {}
for key in all_artcile_data:
    n += 1
    article = all_artcile_data[key]
    if type(article["title"]) == str:
        title = article["title"].strip().lower()
        title_tf[key] = {title_data[w.lemma_] : tf_title[n][title_data[w.lemma_]] for w in nlp(title) if w.lemma_ not in nlp.Defaults.stop_words}
    if type(article["abstract"]) == str:
        abstract = article["abstract"].strip().lower()
        asb_tf[key] = {abs_data[w.lemma_] : tf_abs[n][abs_data[w.lemma_]] for w in nlp(abstract) if w.lemma_ not in nlp.Defaults.stop_words}

In [20]:
open("DATA/P3/title_tf.json","w").write(json.dumps(title_tf))
open("DATA/P3/asb_tf.json","w").write(json.dumps(asb_tf))

20456427

In [22]:
import spacy
nlp = spacy.load("en_core_web_sm")
flatten = lambda x : [item for subitem in x for item in subitem]
not_lemma = " ".join([all_artcile_data[key]["abstract"].lower().strip() for key in all_artcile_data if (type(all_artcile_data[key]["abstract"]) == str)])
s = " ".join(flatten([[w.lemma_ for w in nlp(all_artcile_data[key]["abstract"].lower().strip())] for key in all_artcile_data if type(all_artcile_data[key]["abstract"]) == str]))

In [25]:
open("./fasttext/fasttext_not_lemma_data.txt","w").write(not_lemma)
open("./fasttext/fasttext_data.txt","w").write(s)

10493895

In [37]:
i = 0
def nx():
    global i
    i += 1
    return i
from nltk.tokenize import word_tokenize
abstract_not_lemma = defaultdict(lambda : nx())
for key in all_artcile_data:
    article = all_artcile_data[key]
    if type(article["abstract"]) == str:
        abstract = article["abstract"].strip().lower()
        for w in word_tokenize(abstract):
            abstract_not_lemma[w]
open("DATA/P3/abstract_not_lemma.json","w").write(json.dumps(abstract_not_lemma))

1061377

In [38]:
abs_data = json.load(open("DATA/P3/abstract_not_lemma.json","r"))
import numpy as np
freq_matrix_abstract = np.zeros((len(all_artcile_data)+2,len(abs_data)+2))
n = 0
for key in all_artcile_data:
    n += 1
    article = all_artcile_data[key]
    if type(article["abstract"]) == str:
        abstract = article["abstract"].strip().lower()
        for w in word_tokenize(abstract):
            freq_matrix_abstract[n][abs_data[w]] += 1

M = len(all_artcile_data)
sum_matrix = (freq_matrix_abstract > 0).sum(axis = 0)
idf_abstract = {int(abs_data[key]) : (np.log2(M/sum_matrix[abs_data[key]]) if sum_matrix[abs_data[key]] != 0 else 0) for key in abs_data}
open("DATA/P3/idf_abstract_not_lemma.json","w").write(json.dumps(idf_abstract))

1520926